# Segmenting and Clustering Neighborhoods in Toronto

# Part1

## Import Table from Wikipedia

In [229]:
import pandas as pd
import numpy as np

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M' 

dfs = pd.read_html(url) #extract table

df = dfs[0] #get first table

## Rename columns

In [230]:
df.rename(columns={'Postal Code':'PostalCode','Neighbourhood':'Neighborhood'}, inplace=True) #change column names

## Take out cases where Borough is not assigned

In [231]:
df = df[df['Borough'] != 'Not assigned'] #take out cases with Borough not assigned
df.reset_index(drop=True, inplace=True)

## Replace missing Neighborhood with Borough

In [ ]:
df['Neighborhood']=df['Neighborhood'].replace('Not assigned', df['Borough']) #replace missing Neighborhood with Borough

In [237]:
df.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,None
1,M4A,North York,None
2,M5A,Downtown Toronto,None
3,M6A,North York,None
4,M7A,Downtown Toronto,None
5,M9A,Etobicoke,None
6,M1B,Scarborough,None
7,M3B,North York,None
8,M4B,East York,None
9,M5B,Downtown Toronto,None


In [238]:
df.shape

(103, 3)

# Part 2

## Import coordinates

In [239]:
df_coord = pd.read_csv("Geospatial_Coordinates.csv")
df_coord.rename(columns={'Postal Code':'PostalCode'}, inplace=True) #change column names
df_coord.head(10)

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


## Merge longitude and  latitude to dataframe

In [240]:
df_merged = pd.merge(df,df_coord, on='PostalCode')
df_merged.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,None,43.753259,-79.329656
1,M4A,North York,None,43.725882,-79.315572
2,M5A,Downtown Toronto,None,43.654260,-79.360636
3,M6A,North York,None,43.718518,-79.464763
4,M7A,Downtown Toronto,None,43.662301,-79.389494
5,M9A,Etobicoke,None,43.667856,-79.532242
6,M1B,Scarborough,None,43.806686,-79.194353
7,M3B,North York,None,43.745906,-79.352188
8,M4B,East York,None,43.706397,-79.309937
9,M5B,Downtown Toronto,None,43.657162,-79.378937


# Part 3

## Consider only Toronto Boroughs

In [241]:
df_toronto = df_merged[df_merged['Borough'].str.contains("Toronto")]
df_toronto.reset_index(drop=True, inplace=True)
df_toronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,None,43.654260,-79.360636
1,M7A,Downtown Toronto,None,43.662301,-79.389494
2,M5B,Downtown Toronto,None,43.657162,-79.378937
3,M5C,Downtown Toronto,None,43.651494,-79.375418
4,M4E,East Toronto,None,43.676357,-79.293031


### Import necesary libraries

In [242]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


## Map Toronto Neighborhoods

In [243]:
address = 'Toronto, Canada'

latitude = 43.7166
longitude = -79.3407
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  

map_toronto

The geograpical coordinate of Toronto are 43.7166, -79.3407.


## Cluster Neighborhoods

In [244]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = df_toronto.loc[:,['Latitude','Longitude']] #consider only longitude and latitude
                                 
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 1, 0, 0, 3, 0, 2])

In [245]:
# add clustering labels to dataframe
df_toronto.insert(0, 'Cluster Labels', kmeans.labels_)
df_toronto.head()

,Cluster Labels,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,0,M5A,Downtown Toronto,None,43.654260,-79.360636
1,0,M7A,Downtown Toronto,None,43.662301,-79.389494
2,0,M5B,Downtown Toronto,None,43.657162,-79.378937
3,0,M5C,Downtown Toronto,None,43.651494,-79.375418
4,1,M4E,East Toronto,None,43.676357,-79.293031


## Generate clusterized map

In [246]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Neighborhood'], df_toronto['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters